In [8]:

import pandas as pd
import numpy as np
import seaborn as sns
import pandas_datareader as pdr
from datetime import datetime
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import tensorflow as tf

In [9]:
from tensorflow.python.keras.engine.sequential import Sequential

from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM ,Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from matplotlib import pyplot
%matplotlib inline

In [10]:
df= pd.read_csv('A:\OverallDataset\Chandigarh\OverallChandigarh.csv')

In [11]:
df

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NOx (ppb),SO2 (ug/m3),CO (mg/m3),Ozone (ug/m3)
0,06-08-2019 13:00,06-08-2019 14:00,NaN,39.18,10.26,1.34,0.43,36.47
1,06-08-2019 14:00,06-08-2019 15:00,NaN,48.52,0.00,47.47,0.45,41.22
2,06-08-2019 15:00,06-08-2019 16:00,NaN,NaN,NaN,NaN,NaN,NaN
3,06-08-2019 16:00,06-08-2019 17:00,NaN,NaN,NaN,NaN,NaN,NaN
4,06-08-2019 17:00,06-08-2019 18:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
56642,31-03-2023 19:00,31-03-2023 20:00,21.50,62.75,148.95,3.38,2.75,2.45
56643,31-03-2023 20:00,31-03-2023 21:00,42.75,136.00,159.38,4.57,2.98,2.12
56644,31-03-2023 21:00,31-03-2023 22:00,61.00,159.25,149.40,4.03,2.97,1.60
56645,31-03-2023 22:00,31-03-2023 23:00,51.75,144.25,137.05,3.90,3.03,1.60


In [12]:
import pandas as pd


# Function to check if a column contains string values
def contains_strings(column):
    return any(isinstance(value, str) for value in column)

# Iterate through the columns and check for string values
for column in df.columns:
    if contains_strings(df[column]):
        print(f"Column '{column}' contains string values.")
    else:
        print(f"Column '{column}' does not contain string values.")


Column 'From Date' contains string values.
Column 'To Date' contains string values.
Column 'PM2.5 (ug/m3)' does not contain string values.
Column 'PM10 (ug/m3)' does not contain string values.
Column 'NOx (ppb)' does not contain string values.
Column 'SO2 (ug/m3)' does not contain string values.
Column 'CO (mg/m3)' does not contain string values.
Column 'Ozone (ug/m3)' does not contain string values.


In [13]:
import pandas as pd

# Assuming your DataFrame is named 'df'
df['From Date'] = pd.to_datetime(df['From Date'], format="%d-%m-%Y %H:%M")
df['To Date'] = pd.to_datetime(df['To Date'], format="%d-%m-%Y %H:%M")

# Verify the data types of the columns
print(df.dtypes)


# Now 'From Date' and 'To Date' are datetime data types

From Date        datetime64[ns]
To Date          datetime64[ns]
PM2.5 (ug/m3)           float64
PM10 (ug/m3)            float64
NOx (ppb)               float64
SO2 (ug/m3)             float64
CO (mg/m3)              float64
Ozone (ug/m3)           float64
dtype: object


In [14]:
import pandas as pd
import numpy as np


# Check for missing values in 'PM2.5 (ug/m3)' and 'PM10 (ug/m3)' columns
missing_pm25 = df['PM2.5 (ug/m3)'].isnull().sum()
missing_pm10 = df['PM10 (ug/m3)'].isnull().sum()

if missing_pm25 > 0:
    print(f"Number of missing values in 'PM2.5 (ug/m3)': {missing_pm25}")

if missing_pm10 > 0:
    print(f"Number of missing values in 'PM10 (ug/m3)': {missing_pm10}")

# Check for outliers in 'PM2.5 (ug/m3)' and 'PM10 (ug/m3)' columns
z_scores_pm25 = np.abs((df['PM2.5 (ug/m3)'] - df['PM2.5 (ug/m3)'].mean()) / df['PM2.5 (ug/m3)'].std())
z_scores_pm10 = np.abs((df['PM10 (ug/m3)'] - df['PM10 (ug/m3)'].mean()) / df['PM10 (ug/m3)'].std())

outliers_pm25 = (z_scores_pm25 > 3)
outliers_pm10 = (z_scores_pm10 > 3)

if outliers_pm25.any():
    print(f"Outliers detected in 'PM2.5 (ug/m3)':")
    print(df[outliers_pm25])

if outliers_pm10.any():
    print(f"Outliers detected in 'PM10 (ug/m3)':")
    print(df[outliers_pm10])

# Handle missing values and outliers as needed
# You can choose to remove or impute missing values, and handle outliers according to your data analysis requirements.

# After handling missing values and outliers, you can proceed with AQI calculations as shown in previous responses.


Number of missing values in 'PM2.5 (ug/m3)': 3472
Number of missing values in 'PM10 (ug/m3)': 2772
Outliers detected in 'PM2.5 (ug/m3)':
                From Date             To Date  PM2.5 (ug/m3)  PM10 (ug/m3)  \
1976  2019-10-27 21:00:00 2019-10-27 22:00:00         429.82        505.50   
1977  2019-10-27 22:00:00 2019-10-27 23:00:00         375.82        538.58   
1978  2019-10-27 23:00:00 2019-10-28 00:00:00         377.27        420.59   
1979  2019-10-28 00:00:00 2019-10-28 01:00:00         634.94        793.01   
1981  2019-10-28 02:00:00 2019-10-28 03:00:00         235.86        303.82   
...                   ...                 ...            ...           ...   
55687 2023-02-20 00:00:00 2023-02-20 01:00:00         200.25        266.75   
55688 2023-02-20 01:00:00 2023-02-20 02:00:00         187.00        244.25   
55707 2023-02-20 20:00:00 2023-02-20 21:00:00         180.75        341.75   
55708 2023-02-20 21:00:00 2023-02-20 22:00:00         197.00        351.50   
56081

In [15]:
# Drop rows with missing values in 'PM2.5 (ug/m3)' and 'PM10 (ug/m3)' columns
df = df.dropna(subset=['PM2.5 (ug/m3)', 'PM10 (ug/m3)'])

# Calculate z-scores for 'PM2.5 (ug/m3)' and 'PM10 (ug/m3)' columns
z_scores_pm25 = np.abs((df['PM2.5 (ug/m3)'] - df['PM2.5 (ug/m3)'].mean()) / df['PM2.5 (ug/m3)'].std())
z_scores_pm10 = np.abs((df['PM10 (ug/m3)'] - df['PM10 (ug/m3)'].mean()) / df['PM10 (ug/m3)'].std())

# Define a threshold for outliers (e.g., 3 standard deviations)
outlier_threshold = 3

# Drop rows with outliers in 'PM2.5 (ug/m3)' and 'PM10 (ug/m3)' columns
df = df[~(z_scores_pm25 > outlier_threshold)]
df = df[~(z_scores_pm10 > outlier_threshold)]

C:\Users\arush\AppData\Local\Temp\ipykernel_12788\1933458044.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~(z_scores_pm10 > outlier_threshold)]


In [16]:
print(df['PM2.5 (ug/m3)'].dtype)
print(df['PM10 (ug/m3)'].dtype)


float64
float64


In [17]:
df

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NOx (ppb),SO2 (ug/m3),CO (mg/m3),Ozone (ug/m3)
551,2019-08-29 12:00:00,2019-08-29 13:00:00,2.58,51.41,31.16,3.74,0.87,8.23
555,2019-08-29 16:00:00,2019-08-29 17:00:00,8.19,84.57,20.34,9.71,0.82,18.74
556,2019-08-29 17:00:00,2019-08-29 18:00:00,7.95,87.73,21.35,8.66,0.90,12.32
644,2019-09-02 09:00:00,2019-09-02 10:00:00,8.06,102.43,NaN,4.28,0.57,26.41
645,2019-09-02 10:00:00,2019-09-02 11:00:00,6.79,92.50,17.02,6.08,0.48,33.64
...,...,...,...,...,...,...,...,...
56642,2023-03-31 19:00:00,2023-03-31 20:00:00,21.50,62.75,148.95,3.38,2.75,2.45
56643,2023-03-31 20:00:00,2023-03-31 21:00:00,42.75,136.00,159.38,4.57,2.98,2.12
56644,2023-03-31 21:00:00,2023-03-31 22:00:00,61.00,159.25,149.40,4.03,2.97,1.60
56645,2023-03-31 22:00:00,2023-03-31 23:00:00,51.75,144.25,137.05,3.90,3.03,1.60


In [18]:
import numpy as np

# Assuming the actual column names in your dataset
pm25_column = 'PM2.5 (ug/m3)'
pm10_column = 'PM10 (ug/m3)'

# Constants for AQI calculation (Indian standard)
pm25_breakpoints = [0, 30, 60, 90, 120, 250]
pm25_aqi_values = [0, 50, 100, 200, 300, 400, 500]

pm10_breakpoints = [0, 50, 100, 250, 350, 430]
pm10_aqi_values = [0, 50, 100, 200, 300, 400, 500]

# Calculate AQI for PM2.5
def calculate_aqi_pm25(pm25):
    for i in range(len(pm25_breakpoints) - 1):
        if pm25_breakpoints[i] <= pm25 <= pm25_breakpoints[i + 1]:
            aqi = ((pm25_aqi_values[i + 1] - pm25_aqi_values[i]) / (pm25_breakpoints[i + 1] - pm25_breakpoints[i])) * (pm25 - pm25_breakpoints[i]) + pm25_aqi_values[i]
            return aqi
    return pm25_aqi_values[-1]

# Calculate AQI for PM10
def calculate_aqi_pm10(pm10):
    for i in range(len(pm10_breakpoints) - 1):
        if pm10_breakpoints[i] <= pm10 <= pm10_breakpoints[i + 1]:
            aqi = ((pm10_aqi_values[i + 1] - pm10_aqi_values[i]) / (pm10_breakpoints[i + 1] - pm10_breakpoints[i])) * (pm10 - pm10_breakpoints[i]) + pm10_aqi_values[i]
            return aqi
    return pm10_aqi_values[-1]

# Calculate AQI for each row in the dataset
def calculate_aqi(row):
    aqi_pm25 = calculate_aqi_pm25(row[pm25_column])
    aqi_pm10 = calculate_aqi_pm10(row[pm10_column])
    return max(aqi_pm25, aqi_pm10)

# Apply the calculate_aqi function to your dataset
df['AQI'] = df.apply(calculate_aqi, axis=1)

# Now, your dataset should have an 'AQI' column with the calculated Air Quality Index.

# You can further save this dataset or use it for analysis.



In [19]:
df

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NOx (ppb),SO2 (ug/m3),CO (mg/m3),Ozone (ug/m3),AQI
551,2019-08-29 12:00:00,2019-08-29 13:00:00,2.58,51.41,31.16,3.74,0.87,8.23,51.41
555,2019-08-29 16:00:00,2019-08-29 17:00:00,8.19,84.57,20.34,9.71,0.82,18.74,84.57
556,2019-08-29 17:00:00,2019-08-29 18:00:00,7.95,87.73,21.35,8.66,0.90,12.32,87.73
644,2019-09-02 09:00:00,2019-09-02 10:00:00,8.06,102.43,NaN,4.28,0.57,26.41,101.62
645,2019-09-02 10:00:00,2019-09-02 11:00:00,6.79,92.50,17.02,6.08,0.48,33.64,92.50
...,...,...,...,...,...,...,...,...,...
56642,2023-03-31 19:00:00,2023-03-31 20:00:00,21.50,62.75,148.95,3.38,2.75,2.45,62.75
56643,2023-03-31 20:00:00,2023-03-31 21:00:00,42.75,136.00,159.38,4.57,2.98,2.12,124.00
56644,2023-03-31 21:00:00,2023-03-31 22:00:00,61.00,159.25,149.40,4.03,2.97,1.60,139.50
56645,2023-03-31 22:00:00,2023-03-31 23:00:00,51.75,144.25,137.05,3.90,3.03,1.60,129.50


In [20]:
#X_train = np.nan_to_num(X_train)  # Replace NaN with 0
#X_test = np.nan_to_num(X_test)


In [21]:
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)


In [22]:
import pandas as pd

# Check for NaN values in the entire DataFrame
nan_df = df.isna()

# Check for NaN values in a specific column
nan_in_column = df['AQI'].isna()


In [23]:
# Filling all the null values with mean.
columns_to_fill = [
    'PM2.5 (ug/m3)',
    'PM10 (ug/m3)',
    'NOx (ppb)',
    'SO2 (ug/m3)',
    'CO (mg/m3)',
    'Ozone (ug/m3)',
]

# Fill null values in the specified columns with their respective means
df[columns_to_fill] = df[columns_to_fill].fillna(df[columns_to_fill].mean())

In [24]:
print(df.isnull().sum())

From Date        0
To Date          0
PM2.5 (ug/m3)    0
PM10 (ug/m3)     0
NOx (ppb)        0
SO2 (ug/m3)      0
CO (mg/m3)       0
Ozone (ug/m3)    0
AQI              0
dtype: int64


In [25]:
# Check for infinite values in the entire DataFrame
is_infinite = np.isinf(df)


In [26]:
# Identify rows with infinite values
rows_with_infinity = df[is_infinite.any(axis=1)]

# Identify columns with infinite values
columns_with_infinity = df.columns[is_infinite.any()]


In [27]:
print("Rows with infinity:")
print(rows_with_infinity)

print("Columns with infinity:")
print(columns_with_infinity)


Rows with infinity:
Empty DataFrame
Columns: [From Date, To Date, PM2.5 (ug/m3), PM10 (ug/m3), NOx (ppb), SO2 (ug/m3), CO (mg/m3), Ozone (ug/m3), AQI]
Index: []
Columns with infinity:
Index([], dtype='object')


In [28]:
import pandas as pd
import numpy as np



# List of columns to check for infinity
columns_to_check = ['PM2.5 (ug/m3)', 'PM10 (ug/m3)',  'NOx (ppb)',
                     'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'AQI']

# Loop through the specified columns and print rows with infinity values
for column_name in columns_to_check:
    # Print rows where the specific column contains positive infinity
    rows_with_infinity = df[df[column_name] == np.inf]

    print("Rows with positive infinity in column {}: ".format(column_name))
    print(rows_with_infinity)

    # Print rows where the specific column contains negative infinity
    rows_with_neg_infinity = df[df[column_name] == -np.inf]

    print("Rows with negative infinity in column {}: ".format(column_name))
    print(rows_with_neg_infinity)


Rows with positive infinity in column PM2.5 (ug/m3): 
Empty DataFrame
Columns: [From Date, To Date, PM2.5 (ug/m3), PM10 (ug/m3), NOx (ppb), SO2 (ug/m3), CO (mg/m3), Ozone (ug/m3), AQI]
Index: []
Rows with negative infinity in column PM2.5 (ug/m3): 
Empty DataFrame
Columns: [From Date, To Date, PM2.5 (ug/m3), PM10 (ug/m3), NOx (ppb), SO2 (ug/m3), CO (mg/m3), Ozone (ug/m3), AQI]
Index: []
Rows with positive infinity in column PM10 (ug/m3): 
Empty DataFrame
Columns: [From Date, To Date, PM2.5 (ug/m3), PM10 (ug/m3), NOx (ppb), SO2 (ug/m3), CO (mg/m3), Ozone (ug/m3), AQI]
Index: []
Rows with negative infinity in column PM10 (ug/m3): 
Empty DataFrame
Columns: [From Date, To Date, PM2.5 (ug/m3), PM10 (ug/m3), NOx (ppb), SO2 (ug/m3), CO (mg/m3), Ozone (ug/m3), AQI]
Index: []
Rows with positive infinity in column NOx (ppb): 
Empty DataFrame
Columns: [From Date, To Date, PM2.5 (ug/m3), PM10 (ug/m3), NOx (ppb), SO2 (ug/m3), CO (mg/m3), Ozone (ug/m3), AQI]
Index: []
Rows with negative infinity in 

In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



# Convert 'From Date' and 'To Date' to timestamps
df['From Date'] = pd.to_datetime(df['From Date'])
df['To Date'] = pd.to_datetime(df['To Date'])

# Extract relevant features from the timestamp
df['Year'] = df['From Date'].dt.year
df['Month'] = df['From Date'].dt.month
# Add more features as needed (e.g., day, hour, minute)

# Define your target variable and features
y = df['AQI']  # Assuming 'AQI' is the column with AQI values
X = df.drop(columns=['AQI', 'From Date', 'To Date'])  # Exclude the AQI and date columns

# Normalize your features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split your dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the GRU model
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    tf.keras.layers.GRU(units=64, return_sequences=True),
    tf.keras.layers.GRU(units=64),
    tf.keras.layers.Dense(1)  # Output layer for AQI prediction
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=13, batch_size=32, validation_data=(X_test, y_test))

# Make predictions on training and testing data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate performance metrics for training data
mae_train = mean_absolute_error(y_train, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

# Calculate performance metrics for testing data
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

# Print the performance metrics
print("Training Metrics:")
print("MAE: ", mae_train)
print("RMSE: ", rmse_train)
print("R2: ", r2_train)

print("\nTesting Metrics:")
print("MAE: ", mae_test)
print("RMSE: ", rmse_test)
print("R2: ", r2_test)


Epoch 1/13
1295/1295 [==============================] - 25s 16ms/step - loss: 9728.1455 - val_loss: 5655.8765
Epoch 2/13
1295/1295 [==============================] - 20s 15ms/step - loss: 3992.4246 - val_loss: 2599.4180
Epoch 3/13
1295/1295 [==============================] - 19s 15ms/step - loss: 1900.8118 - val_loss: 1281.3143
Epoch 4/13
1295/1295 [==============================] - 20s 16ms/step - loss: 910.8419 - val_loss: 589.6017
Epoch 5/13
1295/1295 [==============================] - 20s 15ms/step - loss: 399.1710 - val_loss: 239.6960
Epoch 6/13
1295/1295 [==============================] - 19s 15ms/step - loss: 148.5608 - val_loss: 76.2903
Epoch 7/13
1295/1295 [==============================] - 20s 16ms/step - loss: 43.8998 - val_loss: 18.2169
Epoch 8/13
1295/1295 [==============================] - 19s 15ms/step - loss: 11.1797 - val_loss: 4.4460
Epoch 9/13
1295/1295 [==============================] - 20s 16ms/step - loss: 3.3345 - val_loss: 1.2523
Epoch 10/13
1295/1295 [=========